In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [1]:
!pip install ray==2.10.0
!pip install autogluon.tabular --no-cache-dir -q
!pip install scikit-learn==1.4.0
!pip install autogluon.tabular[catboost]==1.2
!pip install dask[dataframe]

     ━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/66.2 MB 6.1 MB/s eta 0:00:09^C
     ━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/66.2 MB 7.5 MB/s eta 0:00:08
ERROR: Operation cancelled by user
  Using cached catboost-1.2.7.tar.gz (71.5 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached graphviz-0.20.3-py3-none-any.whl (47 kB)
  Using cached plotly-5.24.1-py3-none-any.whl (19.1 MB)
  Using cached tenacity-9.0.0-py3-none-any.whl (28 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for catboost (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [11866 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build/lib.macosx-10.9-x86_64-cpython-311/catboost
      copying catboost/monoforest.py -> build/lib.macosx-10.9-x86_64-cpython-311/catboost
      copying catboost/plot_helpers.py -> build/lib.macosx-10.9-x86

On va créer des nouvelles features à partir de la date()

In [2]:
def date(Df):
    Df['Policy Start Date'] = pd.to_datetime(Df['Policy Start Date'])
    Df['Year'] = Df['Policy Start Date'].dt.year
    Df['Day'] = Df['Policy Start Date'].dt.day
    Df['Month'] = Df['Policy Start Date'].dt.month
    Df['Month_name'] = Df['Policy Start Date'].dt.month_name()
    Df['Day_of_week'] = Df['Policy Start Date'].dt.day_name()
    Df['Week'] = Df['Policy Start Date'].dt.isocalendar().week
    Df['Year_sin'] = np.sin(2 * np.pi * Df['Year'])
    Df['Year_cos'] = np.cos(2 * np.pi * Df['Year'])
    Df['Month_sin'] = np.sin(2 * np.pi * Df['Month'] / 12)
    Df['Month_cos'] = np.cos(2 * np.pi * Df['Month'] / 12)
    Df['Day_sin'] = np.sin(2 * np.pi * Df['Day'] / 31)
    Df['Day_cos'] = np.cos(2 * np.pi * Df['Day'] / 31)
    Df['Group']=(Df['Year']-2020)*48+Df['Month']*4+Df['Day']//7
    Df.drop('Policy Start Date', axis=1, inplace=True)

    return Df

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train=date(train)
test=date(test)

cat_cols = [col for col in train.columns if train[col].dtype == 'object']
feature_cols = list(test.columns)

#ajout des fréquences pour les colonnes d'objet (1hot encoding en fréquence)
class CategoricalEncoder:
    def __init__(self, train, test):
        self.train = train
        self.test = test

    def frequency_encode(self, cat_cols, feature_cols, drop_org=False):
        combined = pd.concat([self.train, self.test], axis=0, ignore_index=True)

        new_cat_cols = []
        for col in cat_cols:
            freq_encoding = combined[col].value_counts().to_dict()

            self.train[f"{col}_freq"] = self.train[col].map(freq_encoding).astype('float')
            self.test[f"{col}_freq"] = self.test[col].map(freq_encoding).astype('float')

            new_col_name = f"{col}_freq"
            new_cat_cols.append(new_col_name)
            feature_cols.append(new_col_name)
            if drop_org:
                feature_cols.remove(col)

        return self.train, self.test, new_cat_cols, feature_cols
    
encoder = CategoricalEncoder(train, test)
train, test, cat_cols, feature_cols = encoder.frequency_encode(cat_cols, feature_cols, drop_org=True)

train = train[feature_cols + ['Premium Amount']]
test = test[feature_cols]


donnee = train

#on prend une fraction pour mieux tourner
#donnee=donnee.sample(frac=0.5, random_state=42)
donnee = donnee.drop(columns=['id'])

#on passe au log1p pour coller à la métrique de test
donnee['Premium Amount']=np.log1p(donnee['Premium Amount'])


In [5]:
donnee.columns

Index(['Age', 'Annual Income', 'Number of Dependents', 'Health Score',
       'Previous Claims', 'Vehicle Age', 'Credit Score', 'Insurance Duration',
       'Year', 'Day', 'Month', 'Week', 'Year_sin', 'Year_cos', 'Month_sin',
       'Month_cos', 'Day_sin', 'Day_cos', 'Group', 'Gender_freq',
       'Marital Status_freq', 'Education Level_freq', 'Occupation_freq',
       'Location_freq', 'Policy Type_freq', 'Customer Feedback_freq',
       'Smoking Status_freq', 'Exercise Frequency_freq', 'Property Type_freq',
       'Month_name_freq', 'Day_of_week_freq', 'Premium Amount'],
      dtype='object')

In [4]:
donnee.head()

,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Year,Day,...,Occupation_freq,Location_freq,Policy Type_freq,Customer Feedback_freq,Smoking Status_freq,Exercise Frequency_freq,Property Type_freq,Month_name_freq,Day_of_week_freq,Premium Amount
0,19.0,10049.0,1.0,22.598761,2.0,17.0,372.0,5.0,2023,23,...,470636.0,663201.0,669475.0,625952.0,996268.0,510693.0,667500.0,162307.0,284861.0,7.962067
1,39.0,31678.0,3.0,15.569731,1.0,12.0,694.0,2.0,2023,12,...,NaN,668067.0,665822.0,629122.0,1003732.0,498230.0,667500.0,164442.0,287191.0,7.302496
2,23.0,25602.0,3.0,47.177549,1.0,14.0,NaN,3.0,2023,30,...,470636.0,668732.0,669475.0,614826.0,1003732.0,510693.0,667500.0,165556.0,284861.0,6.342121
3,21.0,141855.0,2.0,10.938144,1.0,0.0,367.0,1.0,2024,12,...,NaN,668067.0,664703.0,625952.0,1003732.0,491143.0,666022.0,164442.0,287424.0,6.641182
4,21.0,39651.0,1.0,20.376094,0.0,8.0,598.0,4.0,2021,1,...,470636.0,668067.0,669475.0,625952.0,1003732.0,510693.0,667500.0,162307.0,287424.0,7.612337


In [7]:
#pour vérifier qu'on s'entraine pas sur des NaN 
donnee = donnee.dropna(subset=['Premium Amount'])

In [8]:
donnee.shape

(1200000, 32)

In [20]:
from autogluon.tabular import TabularPredictor
import time

predictor = TabularPredictor(path='/kag/auto',label='Premium Amount',
                              problem_type = 'regression',
                              eval_metric = 'root_mean_squared_error',
                              )

#TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/kag/auto/ds_sub_fit/sub_fit_ho")
predictor.fit(train_data= donnee,
                       presets='best_quality',
                       time_limit = 25000,
                       #modifier la time_limit pour les tests
                       num_bag_folds = 7,
                       num_stack_levels = 4,
                       auto_stack = True,
                       dynamic_stacking=True
                       )
predictor.leaderboard()

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       9.73 GB / 12.67 GB (76.8%)
Disk Space Avail:   74.02 GB / 107.72 GB (68.7%)
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=4, num_bag_folds=7, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacked overfitting.
	Running DyStack for up to 6250s of the 25000s of remaining time (25%).
		Context path: "/kag/auto/ds_sub_fit/sub_fit_ho"
(

(_ray_fit pid=11573) [1000]	valid_set's rmse: 1.05411
(_ray_fit pid=11572) [2000]	valid_set's rmse: 1.04721 [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)


(_ray_fit pid=11572) 	Ran out of time, early stopping on iteration 2293. Best iteration is:
(_ray_fit pid=11572) 	[1899]	valid_set's rmse: 1.0472


(_ray_fit pid=13137) [1000]	valid_set's rmse: 1.04512 [repeated 2x across cluster]
(_ray_fit pid=13108) [2000]	valid_set's rmse: 1.0493 [repeated 2x across cluster]


(_ray_fit pid=13137) 	Ran out of time, early stopping on iteration 2216. Best iteration is: [repeated 2x across cluster]
(_ray_fit pid=13137) 	[2216]	valid_set's rmse: 1.04464 [repeated 2x across cluster]


(_ray_fit pid=14687) [1000]	valid_set's rmse: 1.0488 [repeated 2x across cluster]
(_ray_fit pid=14658) [2000]	valid_set's rmse: 1.05127 [repeated 2x across cluster]


(_ray_fit pid=14658) 	Ran out of time, early stopping on iteration 2220. Best iteration is: [repeated 2x across cluster]
(_ray_fit pid=14658) 	[2152]	valid_set's rmse: 1.05121 [repeated 2x across cluster]


(_ray_fit pid=16221) [1000]	valid_set's rmse: 1.05079 [repeated 2x across cluster]
(_ray_fit pid=16221) [2000]	valid_set's rmse: 1.05041


(_dystack pid=10992) 	-1.0492	 = Validation score   (-root_mean_squared_error)
(_dystack pid=10992) 	1334.72s	 = Training   runtime
(_dystack pid=10992) 	328.26s	 = Validation runtime
(_dystack pid=10992) Fitting model: LightGBM_BAG_L1 ... Training model for up to 226.66s of the 4804.26s of remaining time.
(_ray_fit pid=14687) 	Ran out of time, early stopping on iteration 2212. Best iteration is:
(_ray_fit pid=14687) 	[2087]	valid_set's rmse: 1.04812
(_dystack pid=10992) 	Fitting 7 child models (S1F1 - S1F7) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=6.75%)
(_ray_fit pid=17362) 	Ran out of time, early stopping on iteration 250. Best iteration is:
(_ray_fit pid=17362) 	[215]	valid_set's rmse: 1.04983
(_ray_fit pid=17633) 	Ran out of time, early stopping on iteration 254. Best iteration is: [repeated 2x across cluster]
(_ray_fit pid=17633) 	[252]	valid_set's rmse: 1.04587 [repeated 2x across cluster]
(_ray_fit pid=17919) 	Ran out of time, earl

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L6,-1.044394,root_mean_squared_error,766.579923,17940.447446,0.060374,3.264580,6,True,22
1,WeightedEnsemble_L3,-1.044482,root_mean_squared_error,636.789352,9602.466743,0.046203,0.719115,3,True,10
2,WeightedEnsemble_L4,-1.044508,root_mean_squared_error,694.938521,13475.682417,0.090362,1.169780,4,True,14
3,WeightedEnsemble_L5,-1.044515,root_mean_squared_error,746.314260,16625.450210,0.061561,0.792369,5,True,18
4,LightGBM_BAG_L4,-1.044535,root_mean_squared_error,701.999199,13619.903363,7.151040,145.390726,4,True,16
5,LightGBM_BAG_L2,-1.044539,root_mean_squared_error,585.042207,5027.198957,12.117283,184.379580,2,True,8
6,LightGBM_BAG_L3,-1.044546,root_mean_squared_error,646.222691,9753.210321,9.479542,151.462693,3,True,12
7,LightGBM_BAG_L5,-1.044609,root_mean_squared_error,754.593929,16768.596400,8.341230,143.938559,5,True,20
8,LightGBMXT_BAG_L3,-1.044871,root_mean_squared_error,660.259337,9878.026357,23.516188,276.278728,3,True,11
9,LightGBMXT_BAG_L2,-1.044920,root_mean_squared_error,600.581443,5184.838905,27.656520,342.019527,2,True,7


In [22]:
X=test.drop(columns=['id'])
batch_size = 10000
predictions = []
c=0
predictor = TabularPredictor.load("/kag/auto")
#utilise une boucle pour le .predict pour voir l'avancé
for i in range(0, len(X), batch_size):
    batch = X.iloc[i:i + batch_size]
    predictions.append(predictor.predict(batch,model='WeightedEnsemble_L6'))#modifier avec le meilleur model de test d'autogluon
    print(c)
    c+=1
y_pred_test = pd.concat(predictions, ignore_index=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79


In [23]:
test['Premium Amount']=np.expm1(y_pred_test)
test1=test[['id','Premium Amount']]
test1.to_csv('submission_autog_25ksec.csv',index=False)